### Quick Run Block

In [1]:
# import torch
# from pytorch_lightning.callbacks import ModelCheckpoint

# # Path to the problematic checkpoint
# checkpoint_path = r"c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\pre_trained_models\reid\dukemtmcreid_resnet50_256_128_epoch_120.ckpt"

# # Allow loading of ModelCheckpoint objects (since PyTorch 2.6 blocks this by default)
# torch.serialization.add_safe_globals([ModelCheckpoint])

# # Load the checkpoint safely
# checkpoint = torch.load(checkpoint_path, map_location="cpu", weights_only=False)

# # Print out keys to inspect conflicting ones
# print("Checkpoint keys:", checkpoint.keys())

# # Remove ModelCheckpoint states (if they exist)
# keys_to_remove = [key for key in checkpoint.keys() if "ModelCheckpoint" in key]
# for key in keys_to_remove:
#     del checkpoint[key]

# # Save the cleaned checkpoint
# fixed_checkpoint_path = checkpoint_path.replace(".ckpt", "_fixed.ckpt")
# torch.save(checkpoint, fixed_checkpoint_path)

# print(f"Fixed checkpoint saved to: {fixed_checkpoint_path}")

## Code

In [2]:
import sys
from pathlib import Path
import os

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\ModelDevelopment\experiments


## Multithreaded Studies Research Findings
- ThreadPoolExecutor works best for CentralPipeline pre-processing step (i.e. everything before run_pose)
- Time taken to process one batch of 32 tracklets: 251 seconds => (251*32)/3600 = 2.35 hours ETA @ 200 images per batch cap, num_threads = 6x3 = 18
- Note on the GPU_SEMAPHORE constant inside ImageFeatureTransformPipeline: this is always capped at 1 so we only offload 1 batch of images to the GPU
- Since this gate is capped at 1, we can control the amount of data offloaded to the GPU via the image_batch_size param to CentralPipeline and only that
- Since caching is implemented, you can try setting the gate to 2 for even faster results, and if the process crashes, keep restarting with use_cache=True

In [3]:
pipeline = CentralPipeline(
  #tracklets_to_process_override=["1"],
  #num_tracklets=1210,
  #num_images_per_tracklet=50,
  input_data_path=DataPaths.TEST_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value,
  single_image_pipeline=False,
  display_transformed_image_sample=False, # NOTE: DO NOT USE. Code is parallelized so we cannot show images anymore. Code breaks, but first one will show if True.
  num_image_samples=1,
  use_cache=True, # Set to false if you encounter data inconsistencies.
  suppress_logging=False,
  
  # --- PARALLELIZATION PARAMS --- These settings are optimal for an NVIDIA RTX 3070 Ti Laptop GPU.
  num_workers=4,            # CRITICAL optimisation param. Adjust accordingly. 6
  tracklet_batch_size=32,   # CRITICAL optimisation param. Adjust accordingly. 32
  image_batch_size=200,     # CRITICAL optimisation param. Adjust accordingly. 200
  num_threads_multiplier=3  # CRITICAL optimisation param. Adjust accordingly. 3. Interpretation: num_threads = num_workers * num_threads_multiplier
  )

2025-03-25 20:06:29 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-03-25 20:06:29 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-03-25 20:06:29 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-03-25 20:06:29 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-03-25 20:06:29 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-03-25 20:06:29 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-03-25 20:06:29 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
2

### To Do
- Investigate why last tracklet is not generated (1210 is skipped, probably something to do with batching)

In [ ]:
pipeline.run_soccernet(
  run_soccer_ball_filter=False,
  generate_features=False,
  run_filter=False,
  run_legible=False,
  run_legible_eval=False,
  run_pose=True,
  run_crops=False,
  run_str=False,
  run_combine=False,
  run_eval=False)

2025-03-25 20:06:29 [INFO] Running the SoccerNet pipeline.
2025-03-25 20:06:29 [INFO] Generating json for pose
2025-03-25 20:06:29 [INFO] Reading legible & illegible results from cache (both global files exist).
2025-03-25 20:06:29 [INFO] Used cached data for pose JSON


Generating pose JSON:   0%|          | 0/1210 [00:00<?, ?it/s]

2025-03-25 20:06:29 [INFO] Completed generating JSON for pose
2025-03-25 20:06:29 [INFO] Detecting pose


Running pose estimation:   0%|          | 0/836 [00:00<?, ?it/s]

2025-03-25 20:07:18 [INFO] [780] Current working directory: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
2025-03-25 20:07:18 [INFO] [780] loading annotations into memory...
2025-03-25 20:07:18 [INFO] [780] Done (t=0.00s)
2025-03-25 20:07:18 [INFO] [964] Current working directory: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
2025-03-25 20:07:18 [INFO] [780] creating index...
2025-03-25 20:07:18 [INFO] [964] loading annotations into memory...
2025-03-25 20:07:18 [INFO] [780] index created!
2025-03-25 20:07:18 [INFO] [964] Done (t=0.00s)
2025-03-25 20:07:18 [INFO] [780] 
2025-03-25 20:07:18 [INFO] [522] Current working directory: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
2025-03-25 20:07:18 [INFO] [964] creating index...
2025-03-25 20:07:18 [INFO] [780] The process cannot access the file because it is being used by another process.
2025-03-25 20:07:18 [INFO] [579] Current working directory: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition